# 1. Importing

In [1]:
import logging
import os
import sys
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.decomposition import FastICA

from CONSTANTS import DEVICE, LOG_ROOT, PROJECT_ROOT, SESSION
from models.gru import AttGRUModel
from module.Common import batch_variable_inst, data_iter, generate_tinsts_binary_label
from module.Optimizer import Optimizer
from preprocessing.AutoLabeling import Probabilistic_Labeling
from preprocessing.datacutter.SimpleCutting import cut_by
from preprocessing.Preprocess import Preprocessor
from representations.sequences.statistics import Sequential_TF
from representations.templates.statistics import (
    Simple_template_TF_IDF,
    Template_TF_IDF_without_clean,
)
from utils.Vocab import Vocab

2024-09-15 06:56:32,631 - AttGRU - SESSION_f56fe5747d539d46f028a40c6b1e8b17 - INFO: Construct logger for Attention-Based GRU succeeded, current working directory: /Users/minhthienlongvo/research/MetaLog, logs will be written in /Users/minhthienlongvo/research/MetaLog/logs
2024-09-15 06:56:33,003 - Preprocessor - SESSION_f56fe5747d539d46f028a40c6b1e8b17 - INFO: Construct logger for MetaLog succeeded, current working directory: /Users/minhthienlongvo/research/MetaLog, logs will be written in /Users/minhthienlongvo/research/MetaLog/logs
2024-09-15 06:56:33,005 - StatisticsRepresentation. - SESSION_f56fe5747d539d46f028a40c6b1e8b17 - INFO: Construct logger for Statistics Representation succeeded, current working directory: /Users/minhthienlongvo/research/MetaLog, logs will be written in /Users/minhthienlongvo/research/MetaLog/logs
2024-09-15 06:56:33,006 - Statistics_Template_Encoder - SESSION_f56fe5747d539d46f028a40c6b1e8b17 - INFO: Construct logger for Statistics Template Encoder succeede

# 2. Custom default params

In [2]:
# Custom params
lstm_hiddens = 128
num_layer = 4
batch_size = 100
drop_out = 0.5
epochs = 10

word2vec_file = "glove.42B.300d.txt"
dim = 300
alpha = 2e-3
beta = 2
gamma = 2e-3


parser = "IBM"
mode = "train"
min_cluster_size = 100
min_samples = 100
reduce_dimension = 50
threshold = 0.5

# 3. Function for updating model

In [3]:
def get_updated_network(old, new, lr, load=False):
    updated_theta = {}
    state_dicts = old.state_dict()
    param_dicts = dict(old.named_parameters())

    for i, (k, v) in enumerate(state_dicts.items()):
        if k in param_dicts.keys() and param_dicts[k].grad is not None:
            updated_theta[k] = param_dicts[k] - lr * param_dicts[k].grad
        else:
            updated_theta[k] = state_dicts[k]
    if load:
        new.load_state_dict(updated_theta)
    else:
        new = put_theta(new, updated_theta)
    return new


def put_theta(model, theta):
    def k_param_fn(tmp_model, name=None):
        if len(tmp_model._modules) != 0:
            for k, v in tmp_model._modules.items():
                if name is None:
                    k_param_fn(v, name=str(k))
                else:
                    k_param_fn(v, name=str(name + "." + k))
        else:
            for k, v in tmp_model._parameters.items():
                if not isinstance(v, torch.Tensor):
                    continue
                tmp_model._parameters[k] = theta[str(name + "." + k)]

    k_param_fn(model)
    return model

# 4. MetaLog class

In [4]:
class MetaLog:
    def __init__(self, vocab, num_layer, hidden_size, drop_out, label2id):
        self.label2id = label2id
        self.vocab = vocab
        self.num_layer = num_layer
        self.hidden_size = hidden_size
        self.batch_size = 128
        self.test_batch_size = 1024
        self.drop_out = drop_out
        self.model = AttGRUModel(vocab, self.num_layer, self.hidden_size, self.drop_out)
        self.bk_model = AttGRUModel(
            vocab, self.num_layer, self.hidden_size, self.drop_out, is_backup=True
        )
        if torch.cuda.is_available():
            self.model = self.model.cuda(DEVICE)
            self.bk_model = self.bk_model.cuda(DEVICE)
        elif hasattr(torch.mps, "is_available") and torch.mps.is_available():
            self.model = self.model.to(DEVICE)
            self.bk_model = self.bk_model.to(DEVICE)
        self.loss = nn.BCELoss()

    def forward(self, inputs, targets):
        tag_logits = self.model(inputs)
        tag_logits = F.softmax(tag_logits, dim=1)
        loss = self.loss(tag_logits, targets)
        return loss

    def bk_forward(self, inputs, targets):
        tag_logits = self.bk_model(inputs)
        tag_logits = F.softmax(tag_logits, dim=1)
        loss = self.loss(tag_logits, targets)
        return loss

    def predict(self, inputs, threshold=None):
        with torch.no_grad():
            tag_logits = self.model(inputs)
            tag_logits = F.softmax(tag_logits, dim=1)
        if threshold is not None:
            probs = tag_logits.detach().cpu().numpy()
            anomaly_id = self.label2id["Anomalous"]
            pred_tags = np.zeros(probs.shape[0])
            for i, logits in enumerate(probs):
                if logits[anomaly_id] >= threshold:
                    pred_tags[i] = anomaly_id
                else:
                    pred_tags[i] = 1 - anomaly_id

        else:
            pred_tags = tag_logits.detach().max(1)[1].cpu()
        return pred_tags, tag_logits

    def evaluate(self, dataset, instances, threshold=0.5):
        logger.info(f"Start evaluating {dataset} by threshold {threshold}")
        with torch.no_grad():
            self.model.eval()
            globalBatchNum = 0
            TP, TN, FP, FN = 0, 0, 0, 0
            tag_correct, tag_total = 0, 0
            for onebatch in data_iter(instances, self.test_batch_size, False):
                tinst = generate_tinsts_binary_label(onebatch, vocab_BGL, False)
                if torch.cuda.is_available():
                    tinst.to_cuda(DEVICE)
                elif hasattr(torch.mps, "is_available") and torch.mps.is_available():
                    tinst.to_mps(DEVICE)
                self.model.eval()
                pred_tags, tag_logits = self.predict(tinst.inputs, threshold)
                for inst, bmatch in batch_variable_inst(
                    onebatch, pred_tags, tag_logits, processor_BGL.id2tag
                ):
                    tag_total += 1
                    if bmatch:
                        tag_correct += 1
                        if inst.label == "Normal":
                            TN += 1
                        else:
                            TP += 1
                    else:
                        if inst.label == "Normal":
                            FP += 1
                        else:
                            FN += 1
                globalBatchNum += 1
            if TP + FP != 0:
                precision = 100 * TP / (TP + FP)
                recall = 100 * TP / (TP + FN)
                f1_score = 2 * precision * recall / (precision + recall)
                # fpr = 100 * FP / (FP + TN)
                logger.info(
                    f"{dataset}: F1 score = {f1_score} | Precision = {precision} | Recall = {recall}"
                )
            else:
                logger.info(
                    f"{dataset}: F1 score = {0} | Precision = {0} | Recall = {0}"
                )
                precision, recall, f1_score = 0, 0, 0
        return precision, recall, f1_score

# 5. Logging

## 5.1. Logging config

In [5]:
logger = logging.getLogger("MetaLog")
logger.setLevel(logging.DEBUG)
console_handler = logging.StreamHandler(sys.stderr)
console_handler.setLevel(logging.DEBUG)
console_handler.setFormatter(
    logging.Formatter(
        "%(asctime)s - %(name)s - " + SESSION + " - %(levelname)s: %(message)s"
    )
)
file_handler = logging.FileHandler(os.path.join(LOG_ROOT, "MetaLog.log"))
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(
    logging.Formatter(
        "%(asctime)s - %(name)s - " + SESSION + " - %(levelname)s: %(message)s"
    )
)
logger.addHandler(console_handler)
logger.addHandler(file_handler)
logger.info(
    f"Construct logger for MetaLog succeeded, current working directory: {os.getcwd()}, logs will be written in {LOG_ROOT}"
)

2024-09-15 06:56:33,042 - MetaLog - SESSION_f56fe5747d539d46f028a40c6b1e8b17 - INFO: Construct logger for MetaLog succeeded, current working directory: /Users/minhthienlongvo/research/MetaLog, logs will be written in /Users/minhthienlongvo/research/MetaLog/logs


## 5.2. Log custom params

In [6]:
logger.info(
    f"Network params: lstm_hiddens = {lstm_hiddens} | num_layer = {num_layer} | drop_out = {drop_out}."
)
logger.info(
    f"Hyper-params: alpha = {alpha} | beta = {beta} | gamma = {gamma} | word2vec_file = {word2vec_file}."
)

2024-09-15 06:56:33,047 - MetaLog - SESSION_f56fe5747d539d46f028a40c6b1e8b17 - INFO: Network params: lstm_hiddens = 128 | num_layer = 4 | drop_out = 0.5.
2024-09-15 06:56:33,048 - MetaLog - SESSION_f56fe5747d539d46f028a40c6b1e8b17 - INFO: Hyper-params: alpha = 0.002 | beta = 2 | gamma = 0.002 | word2vec_file = glove.42B.300d.txt.


# 6. Import dataset

## 6.1. Import BGL dataset

In [7]:
dataset = "BGL"

# # Mark results saving directories.
save_dir = os.path.join(PROJECT_ROOT, "outputs")
prob_label_res_file_BGL = os.path.join(
    save_dir,
    "results/MetaLog/"
    + dataset
    + "_"
    + parser
    + "/prob_label_res/mcs-"
    + str(min_cluster_size)
    + "_ms-"
    + str(min_samples),
)
rand_state_BGL = os.path.join(
    save_dir,
    "results/MetaLog/" + dataset + "_" + parser + "/prob_label_res/random_state",
)

output_model_dir = os.path.join(
    save_dir, "models/MetaLog/" + dataset + "_" + parser + "/model"
)
output_res_dir = os.path.join(
    save_dir, "results/MetaLog/" + dataset + "_" + parser + "/detect_res"
)

# Training, Validating and Testing instances.
template_encoder_BGL = (
    Template_TF_IDF_without_clean(word2vec_file)
    if dataset == "NC"
    else Simple_template_TF_IDF(word2vec_file)
)
processor_BGL = Preprocessor()
train_BGL, _, test_BGL = processor_BGL.process(
    dataset=dataset,
    parsing=parser,
    cut_func=cut_by(0.3, 0., 0.01),
    template_encoding=template_encoder_BGL.present,
)

# Log sequence representation.
sequential_encoder_BGL = Sequential_TF(processor_BGL.embedding)
train_reprs_BGL = sequential_encoder_BGL.present(train_BGL)
for index, inst in enumerate(train_BGL):
    inst.repr = train_reprs_BGL[index]
test_reprs_BGL = sequential_encoder_BGL.present(test_BGL)
for index, inst in enumerate(test_BGL):
    inst.repr = test_reprs_BGL[index]

# Dimension reduction if specified.
transformer_BGL = None
if reduce_dimension != -1:
    start_time = time.time()
    print(f"Start FastICA, target dimension: {reduce_dimension}.")
    transformer_BGL = FastICA(n_components=reduce_dimension)
    train_reprs_BGL = transformer_BGL.fit_transform(train_reprs_BGL)
    for idx, inst in enumerate(train_BGL):
        inst.repr = train_reprs_BGL[idx]
    print(f"Finished at {round(time.time() - start_time, 2)}.")

# Probabilistic labeling.
# Sample normal instances.
train_normal_BGL = [x for x, inst in enumerate(train_BGL) if inst.label == "Normal"]
normal_ids_BGL = train_normal_BGL[: int(0.5 * len(train_normal_BGL))]
label_generator_BGL = Probabilistic_Labeling(
    min_samples=min_samples,
    min_clust_size=min_cluster_size,
    res_file=prob_label_res_file_BGL,
    rand_state_file=rand_state_BGL,
)
labeled_train_BGL = label_generator_BGL.auto_label(train_BGL, normal_ids_BGL)

# Load Embeddings
vocab_BGL = Vocab()
vocab_BGL.load_from_dict(processor_BGL.embedding)

2024-09-15 06:56:33,054 - Statistics_Template_Encoder - SESSION_f56fe5747d539d46f028a40c6b1e8b17 - INFO: Loading word2vec dict from glove.42B.300d.txt.


## 6.2. Import HDFS dataset

In [ ]:
dataset = "HDFS"
# Mark results saving directories.
save_dir = os.path.join(PROJECT_ROOT, "outputs")
prob_label_res_file_HDFS = os.path.join(
    save_dir,
    "results/MetaLog/"
    + dataset
    + "_"
    + parser
    + "/prob_label_res/mcs-"
    + str(min_cluster_size)
    + "_ms-"
    + str(min_samples),
)
rand_state_HDFS = os.path.join(
    save_dir,
    "results/MetaLog/" + dataset + "_" + parser + "/prob_label_res/random_state",
)

# Training, Validating and Testing instances.
template_encoder_HDFS = (
    Template_TF_IDF_without_clean(word2vec_file)
    if dataset == "NC"
    else Simple_template_TF_IDF(word2vec_file)
)
processor_HDFS = Preprocessor()
train_HDFS, _, _ = processor_HDFS.process(
    dataset=dataset,
    parsing=parser,
    cut_func=cut_by(0.3, 0.),
    template_encoding=template_encoder_HDFS.present,
)

# Log sequence representation.
sequential_encoder_HDFS = Sequential_TF(processor_HDFS.embedding)
train_reprs_HDFS = sequential_encoder_HDFS.present(train_HDFS)
for index, inst in enumerate(train_HDFS):
    inst.repr = train_reprs_HDFS[index]

# Dimension reduction if specified.
transformer_HDFS = None
if reduce_dimension != -1:
    start_time = time.time()
    print(f"Start FastICA, target dimension: {reduce_dimension}.")
    transformer_HDFS = FastICA(n_components=reduce_dimension)
    train_reprs_HDFS = transformer_HDFS.fit_transform(train_reprs_HDFS)
    for idx, inst in enumerate(train_HDFS):
        inst.repr = train_reprs_HDFS[idx]
    print(f"Finished at {round(time.time() - start_time, 2)}.")

labeled_train_HDFS = train_HDFS

2024-09-14 13:16:25,782 - Statistics_Template_Encoder - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Loading word2vec dict from glove.42B.300d.txt.
100%|██████████| 1917494/1917494 [01:24<00:00, 22601.49it/s]
2024-09-14 13:18:13,070 - Statistics_Template_Encoder - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Total 1917494 words in glove.42B.300d.txt dict.
2024-09-14 13:18:13,117 - HDFSLoader - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Construct self.logger success, current working directory: /Users/minhthienlongvo/research/MetaLog, logs will be written in /Users/minhthienlongvo/research/MetaLog/logs
2024-09-14 13:18:13,124 - HDFSLoader - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Start load from previous extraction. File path /Users/minhthienlongvo/research/MetaLog/datasets/HDFS/raw_log_seqs.txt
100%|██████████| 575061/575061 [00:03<00:00, 183822.48it/s]
2024-09-14 13:18:16,414 - HDFSLoader - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Extraction finished succ

* ipandport block* * * block* ipandport * ipandport ipandport ipandport ipandport ipandport ipandport block* ipandport ipandport ipandport ipandport block* ipandport ipandport ipandport ipandport * * * block* ipandport * ipandport * ipandport * ipandport ipandport ipandport * ipandport * ipandport ipandport block* ipandport block* ipandport ipandport * * * * ipandport ipandport * ipandport ipandport block* * * 

100%|██████████| 575061/575061 [00:02<00:00, 255770.11it/s]
2024-09-14 13:18:24,860 - Preprocessor - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Train: 165180 Normal, 7338 Anomalous instances.
2024-09-14 13:18:24,861 - Preprocessor - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Test: 393043 Normal, 9500 Anomalous instances.
2024-09-14 13:18:26,233 - Preprocessor - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Update train instances' event-idx mapping.
2024-09-14 13:18:26,575 - Preprocessor - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Embed size: 41 in pre dataset.
2024-09-14 13:18:26,575 - Preprocessor - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Update test instances' event-idx mapping.
2024-09-14 13:18:26,992 - Preprocessor - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Embed size: 46 in pre+post dataset.


Start FastICA, target dimension: 50.
Finished at 19.6.


/Users/minhthienlongvo/research/MetaLog/.venv/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


## 6.3. Aggregate vocab and label2id

In [ ]:
vocab = Vocab()
new_embedding = {}
for key in processor_BGL.embedding.keys():
    new_embedding[key] = processor_BGL.embedding[key]
for key in processor_HDFS.embedding.keys():
    new_embedding[key + 432] = processor_HDFS.embedding[key]
# Load Embeddings
vocab_HDFS = Vocab()
vocab_HDFS.load_from_dict(processor_HDFS.embedding)
vocab.load_from_dict(new_embedding)

metalog = MetaLog(
    vocab=vocab,
    num_layer=num_layer,
    hidden_size=lstm_hiddens,
    drop_out=drop_out,
    label2id=processor_BGL.label2id,
)

2024-09-14 13:19:19,945 - Vocab - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Total words: 50
2024-09-14 13:19:19,948 - Vocab - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: The dim of pretrained embeddings: 300
2024-09-14 13:19:19,957 - Vocab - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Total words: 482
2024-09-14 13:19:19,958 - Vocab - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: The dim of pretrained embeddings: 300
2024-09-14 13:19:19,979 - AttGRU - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: ==== Model Parameters ====
2024-09-14 13:19:19,981 - AttGRU - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Input Dimension: 300
2024-09-14 13:19:19,981 - AttGRU - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Hidden Size: 64
2024-09-14 13:19:19,982 - AttGRU - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Num Layers: 4
2024-09-14 13:19:19,982 - AttGRU - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Dropout: 0.2


2 128
Orthogonal pretrainer loss: 9.31e-03
2 128
Orthogonal pretrainer loss: 5.45e+07


# 7. Training

In [ ]:
# Meta-learning
log = "layer={}_hidden={}_epoch={}".format(num_layer, lstm_hiddens, epochs)
best_model_file = os.path.join(output_model_dir, log + "_best.pt")
last_model_file = os.path.join(output_model_dir, log + "_last.pt")

if not os.path.exists(output_model_dir):
    os.makedirs(output_model_dir)
if mode == "train":
    # Train
    optimizer = Optimizer(
        filter(lambda p: p.requires_grad, metalog.model.parameters()), lr=gamma
    )
    global_step = 0
    best_f1_score = 0
    for epoch in range(epochs):
        metalog.model.train()
        metalog.bk_model.train()
        start = time.strftime("%H:%M:%S")
        logger.info(
            f"Starting epoch: {epoch} | phase: train | start time: {start} | learning rate: {optimizer.lr}."
        )

        batch_num = int(np.ceil(len(labeled_train_HDFS) / float(batch_size)))
        batch_iter = 0
        batch_num_test = int(np.ceil(len(labeled_train_BGL) / float(batch_size)))
        batch_iter_test = 0
        total_bn = max(batch_num, batch_num_test)
        meta_train_loader = data_iter(labeled_train_HDFS, batch_size, True)
        meta_test_loader = data_iter(labeled_train_BGL, batch_size, True)

        for i in range(total_bn):
            optimizer.zero_grad()
            
            # Meta-train on HDFS and BGL
            meta_train_batch = meta_train_loader.__next__()
            meta_test_batch = meta_test_loader.__next__()
            tinst_tr = generate_tinsts_binary_label(meta_train_batch, vocab_HDFS)
            if torch.cuda.is_available():
                tinst_tr.to_cuda(DEVICE)
            elif hasattr(torch.mps, "is_available") and torch.mps.is_available():
                tinst_tr.to_mps(DEVICE)
            loss = metalog.forward(tinst_tr.inputs, tinst_tr.targets)
            loss_value = loss.data.cpu().numpy()
            loss.backward(retain_graph=True)
            batch_iter += 1
            if torch.cuda.is_available():
                metalog.bk_model = (
                    get_updated_network(metalog.model, metalog.bk_model, alpha)
                    .train()
                    .cuda()
                )
            elif hasattr(torch.mps, "is_available") and torch.mps.is_available():
                metalog.bk_model = (
                    get_updated_network(metalog.model, metalog.bk_model, alpha)
                    .train()
                    .to(DEVICE)
                )
            else:
                metalog.bk_model = get_updated_network(
                    metalog.model, metalog.bk_model, alpha
                ).train()
            
            # Meta-test on BGL
            tinst_test = generate_tinsts_binary_label(meta_test_batch, vocab_BGL)
            if torch.cuda.is_available():
                tinst_test.to_cuda(DEVICE)
            elif hasattr(torch.mps, "is_available") and torch.mps.is_available():
                tinst_test.to_mps(DEVICE)
            loss_te = beta * metalog.bk_forward(
                tinst_test.inputs, tinst_test.targets
            )
            loss_value_te = loss_te.data.cpu().numpy() / beta
            loss_te.backward()
            batch_iter_test += 1
            
            # Aggregate the gradients and update the model.
            optimizer.step()
            global_step += 1
            if global_step % 500 == 0:
                logger.info(
                    f"Step: {global_step} | Epoch: {epoch} | Meta-train loss: {loss_value} | Meta-test loss: {loss_value_te}."
                )
            if batch_iter == batch_num:
                meta_train_loader = data_iter(labeled_train_HDFS, batch_size, True)
                batch_iter = 0
            if batch_iter_test == batch_num_test:
                meta_test_loader = data_iter(labeled_train_BGL, batch_size, True)
                batch_iter_test = 0

        if train_BGL:
            metalog.evaluate("Train", train_BGL + train_HDFS)
            
        if test_BGL:
            _, _, f1_score = metalog.evaluate("Test", test_BGL)

            if f1_score > best_f1_score:
                logger.info(
                    f"Exceed best F1 score: history = {best_f1_score}, current = {f1_score}."
                )
                torch.save(metalog.model.state_dict(), best_model_file)
                best_f1_score = f1_score

        logger.info(f"Training epoch {epoch} finished.")
        torch.save(metalog.model.state_dict(), last_model_file)

2024-09-14 13:19:25,868 - MetaLog - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Starting epoch: 0 | phase: train | start time: 13:19:25 | learning rate: [0.002].
/Users/minhthienlongvo/research/MetaLog/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:935: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:494.)
  param_grad = param.grad
2024-09-14 13:31:46,294 - MetaLog - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: Step: 500 | Epoch: 0 | Meta-train loss: 0.0019326373003423214 | Meta-tes

# 8. Evaluate

In [ ]:
if os.path.exists(last_model_file):
    logger.info("=== Final Model ===")
    metalog.model.load_state_dict(torch.load(last_model_file))
    metalog.evaluate("Last model on Test BGL", test_BGL)
if os.path.exists(best_model_file):
    logger.info("=== Best Model ===")
    metalog.model.load_state_dict(torch.load(best_model_file))
    metalog.evaluate("Best model on Test BGL", test_BGL)
logger.info("All Finished!")

2024-09-14 20:36:38,215 - MetaLog - SESSION_e1dd795244c3980b7a9b81fcf515ade8 - INFO: === Final Model ===
/var/folders/45/0lfy6jf14g3cptljv3x_x53r0000gn/T/ipykernel_27776/2143043944.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on Gi

RuntimeError: Error(s) in loading state_dict for AttGRUModel:
	size mismatch for word_embed.weight: copying a param with shape torch.Size([482, 200]) from checkpoint, the shape in current model is torch.Size([482, 300]).
	size mismatch for rnn.weight_ih_l0: copying a param with shape torch.Size([192, 200]) from checkpoint, the shape in current model is torch.Size([192, 300]).
	size mismatch for rnn.weight_ih_l0_reverse: copying a param with shape torch.Size([192, 200]) from checkpoint, the shape in current model is torch.Size([192, 300]).